# Data Cleaning Playground Notebook

This notebook contains random code snippets, thoughts, and ideas for cleaning up the data related to my Splatoon 3 battle data.

## Package Imports and Set Up

In [50]:
import pandas as pd
import json
import requests
import numpy as np

## Read in My Personal Battle Data

In [51]:
battle_df = pd.read_csv('./../data/statink-super64guy.csv', index_col='id')
len(battle_df.index)

327

In [52]:
print(battle_df['lobby'].isna().sum())
print(battle_df['rule'].isna().sum())
print(battle_df['stage'].isna().sum())
print(battle_df['rank_before'].isna().sum())
print(battle_df['game_version'].isna().sum())

0
0
0
221
0


## General Function to Pop Out JSON Data

In [53]:
def pop_general_json_data(df, col_name):
    for i in df.index:
        df.loc[i, col_name] = eval(df[col_name][i])['name']['en_US']

In [54]:
col_names = ['lobby', 'rule', 'stage', 'game_version']

for name in col_names:
    pop_general_json_data(battle_df, name)

battle_df.to_csv('test.csv', index=True)

## Function for Popping Timestamps

In [55]:
def pop_timestamp_json_data(df, col_name):
    for i in df.index:
        df.loc[i, col_name] = eval(df[col_name][i])['time']

In [56]:
col_names = ['start_at', 'end_at']

for name in col_names:
    pop_timestamp_json_data(battle_df, name)

battle_df.to_csv('test.csv', index=True)

## Function for Popping Weapons

In [57]:
def pop_weapon_data(df):
    main_list = []
    sub_list = []
    special_list = []
    for i in df.index:
        tmp_dict = eval(df.loc[i, 'weapon'])
        main_list.append(tmp_dict['name']['en_US'])
        sub_list.append(tmp_dict['sub']['name']['en_US'])
        special_list.append(tmp_dict['special']['name']['en_US'])
    if len(main_list) == len(sub_list) == len(special_list):
        df['main_weapon'] = main_list
        df['sub_weapon'] = sub_list
        df['special_weapon'] = special_list

pop_weapon_data(battle_df)
battle_df.to_csv('test.csv', index=True)

## Update Rule to Remove Lobby

In [58]:
set(battle_df['lobby'].to_list())

{'Anarchy Battle (Open)',
 'Anarchy Battle (Series)',
 'Private Battle',
 'Regular Battle',
 'Splatfest (Open)'}

In [49]:
def update_lobby(df):
    for i in df.index:
        if df['lobby'][i] == 'Regular Battle':
            df.loc[i, 'rule'] = df['rule'][i] + ' (Regular)'
        elif df['lobby'][i] == 'Anarchy Battle (Open)':
            df.loc[i, 'rule'] = df['rule'][i] + ' (Open)'
        elif df['lobby'][i] == 'Anarchy Battle (Series)':
            df.loc[i, 'rule'] = df['rule'][i] + ' (Series)'
        elif df['lobby'][i] == 'Private Battle':
            df.loc[i, 'rule'] = df['rule'][i] + ' (Private)'
        elif df['lobby'][i] == 'Splatfest (Open)' and df['rule'][i] != 'Tricolor Turf War':
            df.loc[i, 'rule'] = df['rule'][i] + ' (Splatfest Open)'
        elif df['lobby'][i] == 'Splatfest (Series)':
            df.loc[i, 'rule'] = df['rule'][i] + ' (Splatfest Series)'

update_lobby(battle_df)
battle_df.to_csv('test.csv', index=True)